In [5]:
conda install -c conda-forge glpk

Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda





Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\emok\Anaconda3

  added / updated specs:
    - glpk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    glpk-5.0                   |       h8ffe710_0         3.3 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be INSTALLED:

  glpk               conda-forge/win-64::glpk-5.0-h8ffe710_0 None




glpk-5.0             | 3.3 MB    |            |   0% 
glpk-5.0             | 3.3 MB    |            |   0% 
glpk-5.0             | 3.3 MB    | 7          |   7% 
glpk-5.0             | 3.3 MB    | #2         |  12% 
glpk-5.0             | 3.3 MB    | #9         |  20% 
glpk-5.0             | 3.3 MB    | ##5        |  25% 
glpk-5.0             | 3.3 MB    | ###2       |  33% 
glp

In [4]:
pip install pyomo

Note: you may need to restart the kernel to use updated packages.


In [67]:
import numpy as np
import pandas as pd

from pyomo.environ import *

Static model, when alpha = 0

In [68]:
# Load data into a Pandas DataFrame
data = pd.read_csv(r'D:\Polit\Seminar (Energy Econ)\dCFdata.csv')

# Create a Pyomo model
model = ConcreteModel()

# Define the sets
model.hours = Set(initialize=data.index, doc='Set of hours')

# Define the variables
model.N_pv = Var(domain=NonNegativeReals, initialize=0)
model.N_on = Var(domain=NonNegativeReals, initialize=0)
model.N_of = Var(domain=NonNegativeReals, initialize=0)

# Define the parameters
model.k_ipv = Param(default=40)
model.k_ion = Param(default=30)
model.k_iof = Param(default=46)
model.CF_pv = Param(model.hours, initialize=data['CF_pv'].to_dict())
model.CF_on = Param(model.hours, initialize=data['CF_on'].to_dict())
model.CF_of = Param(model.hours, initialize=data['CF_of'].to_dict())

# Define the constraint
model.d = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of >= data['d'][h])

# Define the objective function
model.Z = Objective(expr=model.k_ipv * model.N_pv +
                   model.k_ion * model.N_on +
                   model.k_iof * model.N_of)

# Solve the optimization problem
solver = SolverFactory('glpk')
solver.solve(model)

# Access the solution
print("N_pv =", model.N_pv())
print("N_on =", model.N_on())
print("N_of =", model.N_of())
print("Z =", model.Z())


N_pv = 32315.4468498913
N_on = 352866.485014811
N_of = 517826.219014595
Z = 35698618.499111354


Dynamic model, when alpha>0

In [80]:
# Load data into a Pandas DataFrame
data = pd.read_csv(r'D:\Polit\Seminar (Energy Econ)\dCFdatavd.csv')

# Create a Pyomo model
model = ConcreteModel()

# Define the sets
model.hours = Set(initialize=data.index, doc='Set of hours')

# Define the variables
model.N_pv = Var(domain=NonNegativeReals, initialize=0)
model.N_on = Var(domain=NonNegativeReals, initialize=0)
model.N_of = Var(domain=NonNegativeReals, initialize=0)
model.V = Var(model.hours, domain=NonNegativeReals, bounds=(0, 0.25*160425), initialize=0)
model.V_charge = Var(model.hours, domain=NonNegativeReals, bounds=lambda model, h:(0, 0.25*data['pv2g'][h]), initialize=0)
model.V_discharge = Var(model.hours, domain=NonNegativeReals, bounds=lambda model, h:(0, 0.25*data['pv2g'][h]), initialize=0)
model.C = Var(model.hours, domain=NonNegativeReals, initialize=0)

# Define the parameters
model.k_ipv = Param(default=40)
model.k_ion = Param(default=30)
model.k_iof = Param(default=46)
model.k_vv2g = Param(default=226.56)
model.eta_charge = Param (default=0.924)
model.eta_discharge = Param(default=0.903)

#Define data
model.CF_pv = Param(model.hours, initialize=data['CF_pv'].to_dict())
model.CF_on = Param(model.hours, initialize=data['CF_on'].to_dict())
model.CF_of = Param(model.hours, initialize=data['CF_of'].to_dict())
model.pv2g = Param(model.hours, initialize=data['pv2g'].to_dict())
model.d_ev = Param(model.hours, initialize=data['d_ev'].to_dict())

# Define the constraint
model.d1 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of + model.V_discharge[h] == data['d'][h])

model.d2 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of == data['d'][h] + model.V_discharge[h] + model.C[h])


# Define a parameter to store the previous hour
model.prev_h = Param(model.hours, initialize=lambda model, h: model.hours.prev(h) if h > model.hours.first() else model.hours.last(), within=model.hours)

model.d3 = Constraint(model.hours, rule=lambda model, h: model.V[h] == model.V[model.prev_h[h]] + model.eta_charge * model.V_charge[h] - model.V_discharge[h]/model.eta_discharge - data['d_ev'][h])

# Define objective function
def obj_expression(model):
    return model.k_ipv * model.N_pv + model.k_ion * model.N_on + model.k_iof * model.N_of + model.k_vv2g * sum(model.V_discharge[h] for h in model.hours)

model.Z = Objective(expr=obj_expression(model)) # Use the defined objective expression with the 'h' variable

# Solve the optimization problem
solver = SolverFactory('glpk')
solver.solve(model)

# Access the solution
print("N_pv =", model.N_pv())
print("N_on =", model.N_on())
print("N_of =", model.N_of())
print("Z =", model.Z())

WARNING (W1002): Setting Var 'V[0]' to a numeric value `50000` outside the
    bounds (0, 40106.25).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
N_pv = 0
N_on = 0
N_of = 0
Z = 0.0
